In [95]:
import pandas as pd
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_row", 999)
import geopandas as gpd
import numpy as np
import requests
import math
import sys
import graphviz 
import csv
import sqlite3
import re
import codecs
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn import tree as tr
import os; os.getcwd()

'/Users/belenmichel/Desktop/MSCAPP/7_Machine Learning/4_FinalProject_Crimes'

In [218]:
# # Pre-ProcessData

def to_date(df, attribute_lst):
    '''
    Converts the data type of a string in the format YYYY-MM-DD to a datetime  
    '''
    for var in attribute_lst:
        df[var] = df[var].astype('datetime64[ns]', errors = 'ignore')
    return df

def to_int(df, attribute_lst):
    '''
    Converts the data type of a string in the format YYYY-MM-DD to a datetime  
    '''
    for var in attribute_lst:
        df[var] = df[var].astype('int64', errors = 'ignore')
    return df

def remove_outliers(df, sd_threshold=3):
    '''
    Takes a dataframe and number of standard deviations to be considered 
    as outlier and returns a df without the observation that have one or
    or more outliers in it's attributes
    input:
        df: pandas data frame
        sd_threshold: standard deviations
    output:
        the new datafrane without outliers
    '''   
    
    return(df[(np.abs(stats.zscore(df.select_dtypes(include=['int']))) < sd_threshold).all(axis=1)])


def fill_nan(df, attributes_lst):
    '''
    Fills the nan with the mean
    input:
        df: pandas data frame
        attributes_lst: list of attributes names
    output:
        dataframe with the replaced nan
    '''   
    for attribute in attributes_lst: 
        df[attribute].fillna(df[attribute].mean(), inplace=True)

def clean_str(col_names):
    '''
    Removes special characters from a string. 
    input:
        col_names: string with column names
    output:
        sting where special characters where removed
    ''' 
    #TODO: replace this ad-hoc function with a re function. 
    #col_names = re.sub(pattern, repl, string, count=0, flags=0) 
    col_names = col_names.replace('.','')
    col_names = col_names.replace('#','')
    col_names = col_names.replace('-','')
    col_names = col_names.replace('(','')
    col_names = col_names.replace(')','')
    col_names = col_names.replace('&','')
    col_names = col_names.replace('/','')
    col_names = col_names.replace('__','')
    return col_names


# # Generate Features/ Predictors


def discretize_variable(df, attribute_lst):
    '''
    Converts continuous variables into discrete variables
    input:
        df: pandas data frame
        attributes_lst: list of attributes names
    output:
        dataframe with the new variables
    ''' 

    for var in attribute_lst:
        new_var = var + 'cat'
        df[new_var] = pd.qcut(df[var], 10, duplicates="drop", labels=False)
    return df

def categorical_to_dummy(df, attribute_lst):
    '''
    Converts categorical variables into one variabel dummies for each category. 
    input:
        df: pandas data frame
        attributes_lst: list of attributes names
    output:
        dataframe with the new variables
    ''' 

    for var in attribute_lst:
        df = pd.get_dummies(df, columns=[var])
    return df

def remove_attribute(df, attribute_lst):
    '''
    Removes attributes in the list from the data frame
    '''     
    return df.drop(attribute_lst, axis=1)

def keep_attribute(df, attribute_lst):
    '''
    Keeps attributes in the list in the data frame
    '''
    for attribute in df.columns:
        if attribute not in attribute_lst:
            df.drop(attribute, axis=1, inplace=True)
    return df

def flag_to_dummy(df, attribute_lst, rename=True):
    '''
    Converts a flag variable to a dummy with 1 for Yes and 0 for No
    '''
    for var in attribute_lst:
        df[var] = df[var].map({'Y': 1, 'N': 0, 'Yes': 1, 'No': 0, 'T': 1, 'F': 0,\
                               'True': 1, 'False': 0, 'OPEN': 1, 'CLOSED': 0})
        if rename:
            new_var_name = var[:-5]
            df.rename(index=str, columns={var: new_var_name}, inplace=True)
    return df

def gender_to_dummy(df, gender_var):  
    '''
    Converts a gender indicative variable to a dummy with 1 for female and 0 for male
    '''
    df[gender_var] = df[gender_var].map({'FEMALE': 1, 'MALE': 0, 'F': 1, 'M': 0})
    df.rename(index=str, columns={gender_var: "FEMALE"}, inplace=True)
    return df




In [161]:
#Cleaning Inmate Profile

file_name = "data/preprocessed/INMT4AA1.csv"
Inmate_Profile = pd.read_csv(file_name) #, nrows=400
Inmate_Profile = Inmate_Profile[Inmate_Profile["INMATE_IS_FELON/MISDEMEANANT"]=="FELON"]
Inmate_Profile = flag_to_dummy(Inmate_Profile, ['ESCAPE_HISTORY_FLAG', 'PRIOR_INCARCERATIONS_FLAG'])
Inmate_Profile = gender_to_dummy(Inmate_Profile, 'INMATE_GENDER_CODE')
Inmate_Profile = categorical_to_dummy(Inmate_Profile, ['INMATE_RACE_CODE'])
#Remove outliers for TOTAL_SENTENCE_LENGTHIN_DAYS, LENGTH_OF_CURRENT_INCARCERATN, LENGTH_OF_RULING_SENTENCES
#Inmate_Profile = remove_outliers(Inmate_Profile, sd_threshold=3)
Inmate_Profile = to_date(Inmate_Profile, ["INMATE_BIRTH_DATE","INMATE_ADMISSION_DATE","FINAL_RULING_PED",\
                              "FINAL_RULING_TRD", "FINAL_RULING_PRD", "FINAL_RULING_MAX_RELEASE_DATE",\
                              "DATE_TRD_&_PRD_LAST_COMPUTED", "DATE_OF_LAST_ARREST_ON_PAROLE"])
#Inmate_Profile["AGE_IN_DAYS"] = Inmate_Profile["INMATE_ADMISSION_DATE"] - Inmate_Profile["INMATE_BIRTH_DATE"]
keep_attribute(Inmate_Profile, ['INMATE_DOC_NUMBER','INMATE_LAST_NAME','INMATE_FIRST_NAME',\
                                'FEMALE',"INMATE_RACE_CODE_BLACK", "INMATE_RACE_CODE_INDIAN",\
                                "INMATE_RACE_CODE_OTHER", "INMATE_RACE_CODE_WHITE","AGE_IN_DAYS"\
                                'ESCAPE_HISTORY', 'PRIOR_INCARCERATIONS',\
                                "INMATE_BIRTH_DATE","INMATE_ADMISSION_DATE","FINAL_RULING_PED",\
                                "FINAL_RULING_TRD", "FINAL_RULING_PRD", "FINAL_RULING_MAX_RELEASE_DATE",\
                                "DATE_TRD_&_PRD_LAST_COMPUTED", "DATE_OF_LAST_ARREST_ON_PAROLE",\
                                "TOTAL_SENTENCE_LENGTHIN_DAYS", "LENGTH_OF_CURRENT_INCARCERATN",\
                                "LENGTH_OF_RULING_SENTENCES"])

new_file_name = file_name[:-4] + '_cleaned.csv'
Inmate_Profile.to_csv(path_or_buf=new_file_name)

#Shall we include?:
#TYPE_OF_LAST_INMATE_MOVEMENT
#MOST_SERIOUS_OFFNSE_CURR_INCAR

#CURRENT_COMMITMENT_PREFIX
#CURRENT_SENTENCE_COMPONENT

#LAST_RULING_PRD_COMMITMENT
#LAST_RULING_PRD_COMPONENT

In [ ]:
#Cleaning SentenceComputation

file_name = "data/preprocessed/INMT4BB1.csv"
Sentence_Computation = pd.read_csv(file_name)
Sentence_Computation = categorical_to_dummy(Sentence_Computation, ['INMATE_COMPUTATION_STATUS_FLAG'])
Sentence_Computation = to_date(Sentence_Computation, ["SENTENCE_BEGIN_DATE_(FOR_MAX)","ACTUAL_SENTENCE_END_DATE",\
                                          "PROJECTED_RELEASE_DATE_(PRD)","PAROLE_DISCHARGE_DATE",\
                                          "PAROLE_SUPERVISION_BEGIN_DATE"])
keep_attribute(Sentence_Computation, ["INMATE_DOC_NUMBER","INMATE_COMMITMENT_PREFIX","INMATE_SENTENCE_COMPONENT",\
                                  "SENTENCE_BEGIN_DATE_(FOR_MAX)","ACTUAL_SENTENCE_END_DATE",\
                                  "PROJECTED_RELEASE_DATE_(PRD)","PAROLE_DISCHARGE_DATE",\
                                  "PAROLE_SUPERVISION_BEGIN_DATE","INMATE_COMPUTATION_STATUS_FLAG_ACTIVE",\
                                  "INMATE_COMPUTATION_STATUS_FLAG_EAR.TERM","INMATE_COMPUTATION_STATUS_FLAG_EXPIRED",\
                                  "INMATE_COMPUTATION_STATUS_FLAG_FUTURE","INMATE_COMPUTATION_STATUS_FLAG_PAROLED",\
                                  "INMATE_COMPUTATION_STATUS_FLAG_POST REL"])

new_file_name = file_name[:-4] + '_cleaned.csv'
Sentence_Computation.to_csv(path_or_buf=new_file_name)



In [209]:
#Cleaning Disciplinary Infraction Charge
file_name = "data/preprocessed/INMT9CF1.csv"
Infraction_Charge = pd.read_csv(file_name, low_memory=False)
Infraction_Charge = categorical_to_dummy(Infraction_Charge, ['DISCIPLINARY_CHARGE_LEVEL'])
Infraction_Charge = flag_to_dummy(Infraction_Charge, ["ACTIVATE_PRIOR_SUSPENSION"], rename=False)


Infraction_Charge
keep_attribute(Infraction_Charge, ["INMATE_DOC_NUMBER", "DISCIPLINARY_CHARGE_LEVEL_APPEAL",\
                                   "DISCIPLINARY_CHARGE_LEVEL_DISCP HEAR OFFC",\
                                   "DISCIPLINARY_CHARGE_LEVEL_UNIT",'ACTIVATE_PRIOR_SUSPENSION'])
#Shall we include?:
#'DISCIPLINARY_APPEAL_DECISION'
#shall we include it?
new_file_name = file_name[:-4] + '_cleaned.csv'
Infraction_Charge.to_csv(path_or_buf=new_file_name)

In [ ]:
#Cleaning Financial_Obligation

file_name = "data/preprocessed/OFNT1BA1.csv"
Financial_Obligation = pd.read_csv(file_name, low_memory=False)
Financial_Obligation = categorical_to_dummy(Financial_Obligation, ["COURT_ORDERED_PAYMENT_TYPE"])
Financial_Obligation = flag_to_dummy(Financial_Obligation, ["PAYEE_ACCOUNT_STATUS_CODE"], rename=False)
#Financial_Obligation = discretize_variable(Financial_Obligation, ["COP_BALANCE"])
keep_attribute(Financial_Obligation, ["OFFENDER_NC_DOC_ID_NUMBER", "COP_COMMITMENT_PREFIX",\
                                      "COP_ACCOUNT_SEQUENCE_NUMBER","COURT_ORDERED_PAYMENT_TYPE_FINE",\
                                      "COURT_ORDERED_PAYMENT_TYPE_COMM. SERVICE FEE","PAYEE_ACCOUNT_STATUS_CODE",\
                                      "COURT_ORDERED_PAYMENT_TYPE_COURT COSTS","COURT_ORDERED_PAYMENT_TYPE_JAIL FEE",\
                                      "COURT_ORDERED_PAYMENT_TYPE_JUDGEMENT","COURT_ORDERED_PAYMENT_TYPE_RESTITUTION",\
                                      "COURT_ORDERED_PAYMENT_TYPE_SUPERVISION FEE", "COP_BALANCEcat"])

new_file_name = file_name[:-4] + '_cleaned.csv'
Financial_Obligation.to_csv(path_or_buf=new_file_name)


In [223]:
#Cleaning Court Commitment

file_name = "data/preprocessed/OFNT3BB1.csv"
Court_Commitment = pd.read_csv(file_name, low_memory=False, nrows=40000)
Court_Commitment
Court_Commitment = categorical_to_dummy(Court_Commitment, ["COURT_ORDERED_PAYMENT_TYPE"])
Court_Commitment = flag_to_dummy(Court_Commitment, ["PAYEE_ACCOUNT_STATUS_CODE"], rename=False)
#Court_Commitment = discretize_variable(Court_Commitment, ["COP_BALANCE"])
keep_attribute(Court_Commitment, ["OFFENDER_NC_DOC_ID_NUMBER", "COP_COMMITMENT_PREFIX",\
                                      "COP_ACCOUNT_SEQUENCE_NUMBER","COURT_ORDERED_PAYMENT_TYPE_FINE",\
                                      "COURT_ORDERED_PAYMENT_TYPE_COMM. SERVICE FEE","PAYEE_ACCOUNT_STATUS_CODE",\
                                      "COURT_ORDERED_PAYMENT_TYPE_COURT COSTS","COURT_ORDERED_PAYMENT_TYPE_JAIL FEE",\
                                      "COURT_ORDERED_PAYMENT_TYPE_JUDGEMENT","COURT_ORDERED_PAYMENT_TYPE_RESTITUTION",\
                                      "COURT_ORDERED_PAYMENT_TYPE_SUPERVISION FEE", "COP_BALANCEcat"])

new_file_name = file_name[:-4] + '_cleaned.csv'
Court_Commitment.to_csv(path_or_buf=new_file_name)



,OFFENDER_NC_DOC_ID_NUMBER,COMMITMENT_PREFIX,COMMITTED_LAST_NAME,COMMITTED_FIRST_NAME,COMMITTED_MIDDLE_NAME,COMMITTED_NAME_SUFFIX,OFFENDER_ADMISSION/INTAKE_DATE,P&P_CASE_INTAKE_DATE,INMATE_COMMITMENT_STATUS_FLAG,COMMITMENT_STATUS_DATE,EARLIEST_SENTENCE_EFFECTIVE_DT,NEW_PERIOD_OF_INCARCERATION_FL,MOST_SERIOUS_OFFENSE_CODE,CO_OF_CONV_MOST_SERIOUS_OFFNSE,TOTAL_SENTENCE_LENGTH,TOTAL_JAIL_CREDITS_(IN_DAYS),NO_RESTITUTION_FLAG,P&P_COMMITMENT_STATUS_FLAG,P&P_COMMITMENT_STATUS_DATE,TOTAL_LENGTH_OF_SUPERVISION,PED_PRIOR_TO_1995_CONVERSION,DATE_OF_LAST_UPDATE,TIME_OF_LAST_UPDATE,NEW_PERIOD_OF_SUPERVISION_FLAG,TYPE_OF_OLD_PE_DATE_CODE
0,1,01,AAL ANUBIA,RACHELL,NaN,NaN,0001-01-01,1992-12-14,NaN,0001-01-01,1992-12-14,NaN,NaN,NaN,NaN,0.0,NaN,EARLY TERM EARLY,1995-05-11,NaN,0001-01-01,0001-01-01,01:00:00,NaN,NaN
1,3,01,AARHUS,STEVEN,CHARLES,NaN,0001-01-01,1988-10-21,NaN,0001-01-01,1988-10-21,NaN,NaN,NaN,NaN,0.0,NaN,EARLY TERM EARLY,1991-08-20,NaN,0001-01-01,0001-01-01,01:00:00,NaN,NaN
2,3,02,AARHUS,STEVEN,CHARLES,NaN,0001-01-01,2015-02-06,NaN,0001-01-01,2015-02-06,N,DWI LEVEL 2,GUILFORD,NaN,0.0,NaN,UNSUPERVED UNSUP,2015-04-29,NaN,0001-01-01,2015-05-05,17:20:07,Y,NaN
3,4,AA,AARON,DAVID,CLETIS,NaN,1983-07-13,1984-04-17,ACTIVE,1983-07-13,1983-07-12,Y,NaN,NaN,NaN,NaN,NaN,NORMAL NORM,1984-04-17,NaN,0001-01-01,0001-01-01,01:00:00,NaN,NaN
4,5,01,AARON,GENE,ALEXANDER,NaN,0001-01-01,1989-08-01,NaN,0001-01-01,1989-08-01,NaN,NaN,NaN,NaN,0.0,NaN,EARLY TERM EARLY,1995-04-17,NaN,0001-01-01,0001-01-01,01:00:00,NaN,NaN
5,5,02,AARON,GENE,ALEXANDER,NaN,0001-01-01,1989-08-01,NaN,0001-01-01,1990-11-05,NaN,NaN,NaN,NaN,0.0,NaN,EARLY TERM EARLY,1995-04-17,NaN,0001-01-01,0001-01-01,01:00:00,NaN,NaN
6,6,AA,AARON,GERALD,NaN,NaN,1973-01-30,0001-01-01,COURT OR,1973-03-28,1973-01-30,Y,NaN,NaN,NaN,NaN,NaN,NaN,0001-01-01,NaN,0001-01-01,0001-01-01,01:00:00,NaN,NaN
7,6,AB,AARON,GERALD,NaN,NaN,1973-04-15,1974-01-14,ACTIVE,1973-04-15,1973-04-11,Y,NaN,NaN,NaN,NaN,NaN,NORMAL NORM,1974-01-14,NaN,1973-08-05,0001-01-01,01:00:00,NaN,REG.PAROLE
8,7,01,AARON,HATTIE,MICHELLE,NaN,0001-01-01,1991-05-22,NaN,0001-01-01,1991-05-22,NaN,NaN,NaN,NaN,0.0,NaN,EARLY TERM EARLY,1994-02-14,NaN,0001-01-01,0001-01-01,01:00:00,NaN,NaN
9,7,02,AARON,HATTIE,MICHELLE,NaN,0001-01-01,1991-05-22,NaN,0001-01-01,1991-05-22,NaN,NaN,NaN,NaN,0.0,NaN,EARLY TERM EARLY,1994-02-14,NaN,0001-01-01,0001-01-01,01:00:00,NaN,NaN


In [224]:
#Cleaning Sentence_Component

file_name = "data/preprocessed/OFNT3CE1.csv"
Sentence_Component = pd.read_csv(file_name, low_memory=False)#, nrows=40000)

Sentence_Component

new_file_name = file_name[:-4] + '_cleaned.csv'
Sentence_Component.to_csv(path_or_buf=new_file_name)


In [225]:
#Cleaning Special_Cond_Sanctions

file_name = "data/preprocessed/OFNT3CE1.csv"
Special_Cond_Sanctions = pd.read_csv(file_name, low_memory=False)#, nrows=40000)

Special_Cond_Sanctions

new_file_name = file_name[:-4] + '_cleaned.csv'
Special_Cond_Sanctions.to_csv(path_or_buf=new_file_name)

In [226]:
#Cleaning Parole_Analyst

file_name = "data/preprocessed/INMT4CA1.csv"
Parole_Analyst = pd.read_csv(file_name, low_memory=False)#, nrows=40000)

Parole_Analyst

new_file_name = file_name[:-4] + '_cleaned.csv'
Parole_Analyst.to_csv(path_or_buf=new_file_name)


In [ ]:
Warrant_Issued = pd.read_csv("data/preprocessed/OFNT9BE1.csv")


In [ ]:
con = sqlite3.connect("crimes.db")
cur = con.cursor()

tables = ['INMT4AA1_cleaned', 'INMT4BB1_cleaned', 'INMT9CF1_cleaned',\
          'OFNT1BA1_cleaned', 'OFNT3BB1_cleaned', 'OFNT3CE1_cleaned',\
          'OFNT3DE1_cleaned', 'INMT4CA1_cleaned']

for table in tables:
    #print(table)
    file_name = "data/preprocessed/{}.csv".format(table)
    col_names = pd.read_csv(file_name, nrows=0).columns
    n_columns = len(col_names)
    col_names = clean_str(', '.join(col_names))
    cur.execute('DROP TABLE IF EXISTS {}'.format(table))
    cur.execute("CREATE TABLE {} ({});".format(table, col_names))
    
    #File contains NULL bytes. That's why I replaced '\0' with ''
    reader = csv.reader(x.replace('\0','') for x in open(file_name))
    for row in reader:
        row = [None if x == '' else x for x in row]
        cur.execute("INSERT INTO {} VALUES ({});".format(table,",".join(['?']*n_columns)), row)

con.commit()
con.close()

In [ ]:
for at in Inmate_Profile.columns:
    print(at)
    print(type(Inmate_Profile[at][0]))

In [198]:
Infraction_Charge.groupby("SUSPENSION_STATUS").size()

SUSPENSION_STATUS
ACTIVATED             329
SUSPENSIONS EXIST    7044
VOID                    1
dtype: int64